# Getting Started with Azure ML Notebooks and Azure Sentinel

---

# Contents

- Introduction
- What is a Jupyter Notebook
- Setting up the notebook environment
- Notebook/MSTICPy configuration
- Querying data from Azure Sentinel
- Testing Threat Intelligence and IP Geolocation lookups
- Further Resources
- FAQs - Frequently Asked Questions

---

# Introduction

This notebook takes you through the basics needed to get started with Azure Machine Learning (ML) Notebooks and Azure Sentinel.

It focuses on getting things set up and basic steps to query data.

After you've finished running this notebook you can go on to look at the following notebooks:

- **A Tour of Cybersec notebook features** - this takes you through some of the basic
  features for CyberSec investigation/hunting available to you in notebooks.
- **Configuring your environment** - this covers all of the configuration options for 
  accessing external cybersec resources


Each topic includes 'learn more' sections to provide you with the resource to deep
dive into each of these topics. We encourage you to work through the notebook from start
to finish.

<div style="color: Black; background-color: Khaki; padding: 5px; font-size: 20px">
<p>Please run the the code cells in sequence. Skipping cells will result in errors.</p>
<div style="font-size: 16px">
<p>If you encounter any unexpected errors please see the FAQ at the end of this notebook.</p>
</div>
</div>

<hr>

---

# What is a Jupyter notebook?

<div style="color: Black; background-color: Gray; padding: 5px; font-size: 20px">
If you're familiar with notebooks, skip this section and go to "Setting up the environment" section.
</div>
<br>

You are currently reading a Jupyter notebook. [Jupyter](http://jupyter.org/) is an interactive
development and data manipulation environment presented in a browser.

A Jupyter notebook is a document
made up of cells that contain interactive code, alongside that code's output,
and other items such as text and images (what you are looking at now is a cell of *Markdown* text).

The name, Jupyter, comes from the core supported programming languages that it supports: **Ju**lia, **Pyt**hon, and **R**.
While you can use any of these languages (and others such as Powershell) we are going to use Python in this notebook.

The majority of the notebooks on the [Azure Sentinel GitHub repo](https://github.com/Azure/Azure-Sentinel-Notebooks)
are written in Python. Whilst there are pros, and cons to each language, Python is a well-established
language that has a large number of materials and libraries well suited for
data analysis and security investigation, making it ideal for our needs.

<details>
  <summary>Learn more...</summary>
  <ul>
    <li><a href=https://infosecjupyterbook.com/introduction.html>The Infosec Jupyter Book</a>
    has more details on the technical working of Jupyter.
    </li>
    <li><a href=https://jupyter.org/documentation>The Jupyter Project documentation</a></li>
  </ul>
</details>

<br>



---

## How to use a Jupyter notebook?

To use a Jupyter notebook you need a Jupyter server that will render the notebook and execute the code within it.
This can take the form of a local [Jupyter installation](https://pypi.org/project/jupyter/),
or a remotely hosted version such as 
[Azure Machine Learning Notebooks](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-run-jupyter-notebooks). 

## Using Azure Machine Learning (ML) Notebooks

If you launched this notebook from Azure Sentinel, you will be running it in an Azure ML workspace.
By default, the notebook is running in the built-in notebook editor. You can also open
and run the notebook in Jupyterlab or Jupyter classic, if these environments are more familiar
to you.

<details>
  <summary>Learn more...</summary>
<p>Although you can view a notebook as a static document (GitHub, for example has a built-in
static notebook renderer), if you want to run the code in a notebook, the notebook
must be attached to a backend process, know as a Jupyter kernel. The kernel
is really where your code is being run and where all of variables and objects
created in the code are held. The browser is just the viewer for this data.
</p>
<p>In Azure ML, the kernel runs on a virtual machine known as an Azure ML Compute.
The Compute instance can support the running of many notebooks simultaneously.</p>
<p>
Usually, the creation/attaching of a kernel for your notebook happens
seamlessly - you don't need to do anything manually. One thing that you
may need to check (especially if you are getting errors or the notebook
doesn't seem to be executing) is the version and state of the kernel.</p>
  <ul>
    <li><a href=https://realpython.com/jupyter-notebook-introduction/>
    Installing and running a local Jupyter server
    </a>
    </li>
  </ul>
</details>

For this notebook we are going to be using Python 3.8 (you can also choose the Python 3.6 kernel).

You can check the kernel name and version is selected by looking at
the drop down in the top left corner of the Workspace window as shown in
the image below.



<img src="https://github.com/Azure/Azure-Sentinel-Notebooks/raw/master/images/AML_kernel.png" height="200px">

This image also shows the active compute instance (to the right).

<img src="https://github.com/Azure/Azure-Sentinel-Notebooks/raw/master/images/AML_compute_kernel.png" height="200px">


If the selected kernel does not show `Azure ML Python 3.8` you can select the correct kernel by clicking on the Kernel drop-down.

<p style="border: solid; padding: 5pt"><b>Note</b>: the notebook works with Python 3.6 or later.
If you are using this notebook in another
Jupyter environment you can choose any kernel that supports Python 3.6 or later
</p>

<p style="border: solid; padding: 5pt; color: white; background-color: DarkOliveGreen"><b>Tip:</b>
Sometimes, your notebook may "hang" or you want to just start over.
To do this you can restart the kernel. Use the "recycle" button in the toolbar
in the upper right of the screen above the notebook.
<br>
You will need to re-run any initialization and authentication cells after doing
this since restarting the kernel wipes all variables and other state.
</p>

<img src="https://github.com/Azure/Azure-Sentinel-Notebooks/raw/master/images/AML_restart_kernel.png" height="200px">

<br>
<details>
  <summary>Troubleshooting...</summary>
  <p>
If you are having trouble getting the notebook running you should review
<a href=https://docs.microsoft.com/en-us/azure/machine-learning/how-to-run-jupyter-notebooks>How to run Juptyer notebooks</a>.
</p>
</details>


---

## Running code

The **cell** below is a code cell (note that it looks different from the
cell you are reading). The current cell is known as a *Markdown* cell
and lets you write text (including HTML) and include static images.

Select the code cell (using mouse or cursor keys) below.
Once selected, you can execute the code in it by clicking the "Play" button in the cell, or by pressing Shift+Enter.

<p style="border: solid; padding: 5pt; color: white; background-color: DarkOliveGreen">
<b>Tip</b>: You can identify which cells are code cells by selecting them.<br>
In Azure ML notebooks and VSCode, code cells have a larger border
on the left side with a "Play" button to execute the cell.<br>
In other notebook environments code and markdown cells will have
different styles but it's usually easy to distinguish them.
</p>


In [1]:
# This is our first code cell, it contains basic Python code.
# You can run a code cell by selecting it and clicking
# the Run button (to the left of the cell), or by pressing Shift + Enter.
# Any output from the code will be displayed directly below it.
print("Congratulations you just ran this code cell")
y = 2 + 2
print("2 + 2 =", y)

Congratulations you just ran this code cell
2 + 2 = 4


Variables set within a code cell persist between cells meaning you can chain cells together.
In this example we're using the value of y from the previous cell.

In [2]:
# Note that output from the last line of a cell is automatically
# sent to the output cell, without needing the print() function.
y + 2

6

Now that you understand the basics we can move onto more complex code.


<details>
  <summary>Learn more about notebooks...</summary>
  <ul>
    <li><a href=https://infosecjupyterbook.com/>The Infosec Jupyter Book</a>
    provides an infosec-specific intro to Python.
    </li>
    <li><a href=https://realpython.com/>Real Python</a>
    is a comprehensive set of Python learnings and tutorials.</li>
  <ul>

</details>


---

# Setting up the notebook environment

To avoid having to type (or paste) a lot of complex and repetitive code into
notebook cells, most notebooks rely on third party libraries (known in the Python
world as "packages").

Before you can use a package in your notebook, you need to do two things:

- install the package (although the Azure ML Compute has most common packages pre-installed)
- import the package (or some part of the package - usually a module/file, a function or a class)

## MSTICPy

We've created a Python package built for Azure Sentinel notebooks - named MSTICPy (pronounced miss-tick-pie).
It is a collection of CyberSecurity tools for data retrieval, analysis, enrichment and visualization.

## Intializing notebooks

At the start of most Azure Sentinel notebooks you will see an initialization cell like the one below.
This cell is specific to the MSTICPy initialization:

- it defines the minimum versions for Python and MSTICPy needed for this notebook
- it then imports and runs the `init_notebook` function.

<details>
    <summary>More about <i>init_notebook</i>...</summary>
    <p>
`init_notebook` does some of the tedious work of importing other packages, 
checking configuration (we'll get to configuration in a moment) and, optionally,
installing other required packages.</p>
</details>
<br>

<div style="bor der: solid; padding: 5pt">
<b>Note:</b>don't be alarmed if you see configuration warnings (such as "Missing msticpyconfig.yaml").<br>
We haven't configured anything yet, so this is expected.
</div>

The first line ensures that the latest version of msticpy is installed.

In [3]:
# import some modules needed in this cell
from IPython.display import display, HTML

REQ_PYTHON_VER="3.6"
REQ_MSTICPY_VER="1.2.3"

display(HTML("Checking upgrade to latest msticpy version"))
%pip install --upgrade --quiet msticpy[azuresentinel]>=$REQ_MSTICPY_VER

# intialize msticpy
from msticpy.nbtools import nbinit
nbinit.init_notebook(
    namespace=globals(),
    extra_imports=["urllib.request, urlretrieve"]
)
pd.set_option("display.html.table_schema", False)

Note: you may need to restart the kernel to use updated packages.


---

# Notebook/MSTICPy Configuration

Once we've done this basic initialization step,
we need to make sure we have some configuration in place.

Some of the notebook components need addtional configuration to connect our Azure Sentinel workspace
as well as to external services (e.g. API keys to retrieve Threat Intelligence data). 

The easiest way to manage the configuration data for these services is to store it in a 
configuration file (`msticpyconfig.yaml`).<br>
The alternative to this is having to type
configuration details and keys in each time you use a notebook - which is
guaranteed to put you off notebooks forever!

<details>
  <summary>Learn more...</summary>
  <p>
  Although you don't need to know these details now, you can find more information here:
  </p>
  <ul>
    <li><a href=https://msticpy.readthedocs.io/en/latest/getting_started/msticpyconfig.html >MSTICPy Package Configuration</a></li>
    <li><a href=https://msticpy.readthedocs.io/en/latest/getting_started/SettingsEditor.html >MSTICPy Settings Editor</a></li>
  </ul>
  <p>If you need a more complete walk-through of configuration, we have a separate notebook to help you:</p>
  <ul>
    <li><a href=https://github.com/Azure/Azure-Sentinel-Notebooks/blob/master/ConfiguringNotebookEnvironment.ipynb >Configuring Notebook Environment</a></li>
    <li>And for the ultimate walk-through of how to configure all your `msticpyconfig.yaml` settings
  see the <a href=https://github.com/microsoft/msticpy/blob/master/docs/notebooks/MPSettingsEditor.ipynb >MPSettingsEditor notebook</a></li>
    <li>The Azure-Sentinel-Notebooks GitHub repo also contains an template `msticpyconfig.yaml`, with commented-out sections
  that may also be helpful in finding your way around the settings if you want to dig into things
  by hand.</li>
  </ul>
</details>
<br>

---

## Configuring Azure Sentinel settings

When you launched this notebook from Azure Sentinel it copied a basic configuration file - `config.json` -
to your workspace folder.<br>
You should be able to see this file in the file browser to the left.<br>
This file contains details about your Azure Sentinel workspace but has
no configuration settings for other external services that we need.

If you didn't have a `msticpyconfig.yaml` file in your workspace folder (which is likely
if this is your first use of notebooks), the `init_notebook` function should have created
one for you and populated it
with the Azure Sentinel workspace data taken from your config.json.

<p style="border: solid; padding: 5pt; color: white; background-color: DarkOliveGreen"><b>Tip:</b>
If you do not see a "msticpyconfig.yaml" file in your user folder, click the refresh button<br>
at the top of the file browser.
</p>

We can check this now by opening the settings editor and view the settings.

<div style="color: Black; background-color: Khaki; border: solid; padding: 5pt;"><b>
You should not have to change anything here unless you need to add
one or more additional workspaces.</b></div>
<p/>
<p style="border: solid; padding: 5pt"><b>Note:</b>
In the Azure ML environment, the settings editor may take 10-20 seconds to appear.<br>
This is a known bug that we are working to fix.
</p>

When you have verified that this looks OK. Click **Save Settings**


<details>
    <summary>Multiple Azure Sentinel workspaces...</summary>
    <p>If you have multiple Azure Sentinel workspaces, you can add
    them in the following configuration cell.</p>
    <p>You can choose to keep one as the default or just delete this entry
    if you always want to name your workspaces explicitly when you 
    connect.
    </p>
</details>

In [4]:
from msticpy.config import MpConfigEdit
import os

mp_conf = "msticpyconfig.yaml"

# check if MSTICPYCONFIG is already an env variable
mp_env = os.environ.get("MSTICPYCONFIG")
mp_conf = mp_env if mp_env and Path(mp_env).is_file() else mp_conf

if not Path(mp_conf).is_file():
    print(
        "No msticpyconfig.yaml was found!",
        "Please check that there is a config.json file in your workspace folder.",
        "If this is not there, go back to the Azure Sentinel portal and launch",
        "this notebook from there.",
        sep="\n"
    )
else:
    mpedit = MpConfigEdit(mp_conf)
    mpedit.set_tab("AzureSentinel")
    display(mpedit)

At this stage you should only see two entries in the Azure Sentinel tab:

- An entry with the name of your Azure Sentinel workspace
- An entry named "Default" with the same settings.

MSTICPy lets you store configurations for multiple Azure Sentinel workspaces
and switch between them. The "Default" entry lets you authenticate
to a specific workspace without having to name it explicitly.

Let's add some additional configuration.

## Adding Threat Intel and GeoIP provider settings

We are going to use [VirusTotal](https://www.virustotal.com) (VT) as an example of a popular threat intelligence source.
To use VirusTotal threat intel lookups you will need a VirusTotal account and API key.

You can sign up for a free account at the
[VirusTotal getting started page](https://developers.virustotal.com/v3.0/reference#getting-started) website.

If you are already a VirusTotal user, you can, of course, use your existing key.

<p style="border: solid; padding: 5pt; color: black; background-color: Khaki">
<b>Warning</b> If you are using a VT enterprise key we do not recommend storing this
in the msticpyconfig.yaml file.<br>
MSTICPy supports storage of secrets in
Azure Key Vault. You can read more about this
<a href=https://msticpy.readthedocs.io/en/latest/getting_started/msticpyconfig.html#specifying-secrets-as-key-vault-secrets >in the MSTICPY docs</a><br>
For the moment, you can sign up for a free acount, until you can take the time to
set up Key Vault storage.
</p>


As well as VirusTotal, we also support a range
of other threat intelligence providers: https://msticpy.readthedocs.io/en/latest/data_acquisition/TIProviders.html
<br><br>

To add the VirusTotal details, run the following cell.

1. Select "VirusTotal" from the **Add prov** drop down
2. Click the **Add** button
3. In the left-side Details panel select **Text** as the Storage option.
4. Paste the API key in the **Value** text box.
5. Click the **Update** button to confirm your changes.

Your changes are not yet saved to your configuration file. To
do this, click on the **Save Settings** button at the bottom of the dialog.

If you are unclear about what anything in the configuration editor means, use the **Help** drop-down. This
has instructions and links to more detailed documentation.


In [5]:
mpedit.set_tab("TI Providers")
mpedit

Our notebooks commonly use IP geo-location information. 
In order to enable this we are going to set up [MaxMind GeoLite2](https://www.maxmind.com)
to provide geolocation lookup services for IP addresses.

GeoLite2 uses a downloaded database which requires an account key to download.
You can sign up for a free account and a license key at 
[The Maxmind signup page - https://www.maxmind.com/en/geolite2/signup](https://www.maxmind.com/en/geolite2/signup).
<br>

<details>
    <summary>Using IPStack as an alernative to GeoLite2...</summary>
    <p>
    For more details see the
    <a href=https://msticpy.readthedocs.io/en/latest/data_acquisition/GeoIPLookups.html >
    MSTICPy GeoIP Providers documentation</a>
    </p>
</details>
<br>

Once, you have an account, run the following cell to add the Maxmind GeopIP Lite details to your configuration.

The procedure is similar to the one we used for VirusTotal:

1. Select the "GeoIPLite" provider from the **Add prov** drop-down
2. Click **Add**
3. Select **Text** Storage and paste the license (API/Auth) key into the text box
4. Click **Update**
5. Click **Save Settings** to write your settings to your configuration.


In [6]:
mpedit.set_tab("GeoIP Providers")
mpedit

## Validate your settings

- click on the **Validate settings** button.

You may see some warnings about missing sections but not about the Azure Sentinel, TIProviders or GeoIP Providers settings.

Click on the **Close** button to hide the validation output.

If you need to make any changes as a result of the Validation,
remember to save your changes by clicking the **Save File** button.

## Loading your saved settings

<h3 stylecolor: Black; background-color: Khaki;i; padding: 5px">
Although your settings are saved they are not yet loaded into MSTICPy. Please run the next cell.</h3>
<hr>

You have saved your settings to a "msticpyconfig.yaml" file on disk. This file<br>
is a YAML file so is easy to read with a text editor.<br>
However, MSTICPy does not automatically reload these settings.

Run the following cell to force it to reload from the new configuration file.


In [7]:
import msticpy
msticpy.settings.refresh_config()


<details>
    <summary>Optional - set a MSTICPYCONFIG environment variable...</summary>

Setting a `MSTICPYCONFIG` environment variable to point to the location of your
your `msticpyconfig.yaml` configuration file lets MSTICPy find this file. By
default, it looks in the current directory only.

Having a MSTICPYCONFIG variable set is especially convenient if you are
using notebooks locally or have a deep or complex folder structure

<b>You may not need this on Azure ML</b> - the
`nb_check` script (in the initialization cell) will automatically set the MSTICPYCONFIG
environment variable if it is not already set. The script will try to find a
`msticpyconfig.yaml` in the current directory or in the root of your AML user folder.

For more detailed instructions on this, please see the *Setting the path to your msticpyconfig.yaml* section
in the <a href=https://github.com/Azure/Azure-Sentinel-Notebooks/blob/master/ConfiguringNotebookEnvironment.ipynb >Configuring Notebook Environment</a>.

<p style="border: solid; padding: 5pt; color: black; background-color: #AA4000">
<b>Warning</b>: If you are storing secrets such as API keys in the file you should<br>
probably opt to store either store this file on the compute instance or use<br.>
Azure Key Vault to store the secrets.<br>
Read more about using KeyVault
<a href=https://msticpy.readthedocs.io/en/latest/getting_started/msticpyconfig.html#specifying-secrets-as-key-vault-secrets >in the MSTICPY docs</a>
</p>

</details>


---

# Querying Data from Azure Sentinel

Now that we have configured the details for your Azure Sentinel workspace, 
we can go ahead and test it. We will do this with `QueryProvider` class from MSTICPy.

The QueryProvider class has one main function:

- querying data from a data source to make it available to view and analyze in the notebook.

Query results are always returned as *pandas* DataFrames. If you are new
to using *pandas* look at the **Introduction to Pandas** section at in
the **A Tour of Cybersec notebook features** notebook.

<details>
    <summary>Other data sources...</summary>
    <p>
    The query provider supports several different data sources - the one we'll use
here is "AzureSentinel".</p>
<p>
Other data sources supported by the `QueryProvider` class include Microsoft Defender for Endpoint,
Splunk, Microsoft Graph API, Azure Resource Graph but these are not covered here.
    </p>
</details>
<br>

Most query providers come with a range of built-in queries
for common data operations. You can also a query provider to run custom queries against
Azure Sentinel data.

Once you've loaded a QueryProvider you'll normally need to authenticate
to the data source (in this case Azure Sentinel).

<details>
    <summary><b>Learn more...</b></summary>
    <p>
    More details on configuring and using QueryProviders:
    </p>
    <ul>
        <li>
 <a href=https://msticpy.readthedocs.io/en/latest/data_acquisition/DataProviders.html#instantiating-a-query-provider >MSTICPy Documentation</a>.</li>
    </ul>
</details>
<br>


## Load a QueryProvider
To start, we are going to load up a QueryProvider
for Azure Sentinel, pass it the 
details for our workspace that we just stored in the msticpyconfig file, and connect.

<div style="border: solid; padding: 5pt"><b>Note:</b>
If you see a warning "Runtime dependency of PyGObject is missing" when loading the<br>
Azure Sentinel driver, please see the FAQ section at the end of this notebook.<br>
The warning does not impact any functionality of the notebooks.
</div>

In [8]:
# Initalize a QueryProvider for Azure Sentinel
qry_prov = QueryProvider("AzureSentinel")

Please wait. Loading Kqlmagic extension...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Authenticate to the Azure Sentinel workspace

Next we need to authenticate The code cell immediately following this section
will start the authentication process. e.When you run the cell it will trigger an Azure authentication sequence.


The connection process will ask us to authenticate to our Azure Sentinel workspace using 
[device authorization](https://docs.microsoft.com/en-us/azure/active-directory/develop/v2-oauth2-device-code)
with our Azure credentials.

Device authorization adds an additional factor to the authentication by generating a
one-time device code that you supply as part of the authentication process.

The process is as follows:

1. generate and display an device code

<img src="https://github.com/Azure/Azure-Sentinel-Notebooks/raw/master/images/device_auth_code.png" height="300px">

In some Jupyter notebook interfaces, step 1 has a clickable button that will copy the code to the clipboard
and then open a browser window to the devicelogin URI.<br>The default AML notebook interface does not
do this. You need to manually select and copy the code to the clipboard, then click on the devicelogin link
where you can paste the code in to start the authentication process.

2. go to http://microsoft.com/devicelogin and paste in the code
3. authenticate with your Azure credentials

<p style="border: sols:id; <br>
1. Tdding: 5pt"><b>Note:</b> the devicelogin URL may be different for government <br>
2. You must use an account that has permissions to the Azure Sentinel workspace.<br>
(<b>Azure Sentinel Reader</b> or <b>Log Analytics Reader</b>)
or national Azure clouds.</p>

<br>

<img src="https://github.com/Azure/Azure-Sentinel-Notebooks/raw/master/images/device_authn.png" height="300px">

4. Once the last box in the previous image is shown you can close that tab and return to the notebook. 
   You should see an image like the one shown below.

<img src="https://github.com/Azure/Azure-Sentinel-Notebooks/raw/master/images/device_auth_complete.png" height="300px">

<br>

<p style="border: solid; padding: 5pt"willb><b>Note:</b>
Once authenticated, you should not need to re-authenticate or open another notebook if you restart the kernel.
You can avoid this by using Azure CLI to create a cached logon on the AML compute(this is described in the FAQ section at the end of the notebook).<br>The Azure CLI logon will cache the token even if you restart kernel.<AML br>
This <i>refresh token</i> is cached on the compute and can be re-used until it
times out.<br>
You will need to re-authenticate if you restart your Compute instance
or switch to a different one.
</p>

In [9]:
# Get the Azure Sentinel workspace details from msticpyconfig
# Loading WorkspaceConfig with no parameters will use the details
# of your "Default" workspace (see the Configuring Azure Sentinel settings section earlier)
# If you want to connect to a specific workspace use this syntax:
#    ws_config = WorkspaceConfig(workspace="WorkspaceName")
# ('WorkspaceName' should be one of the workspaces defined in msticpyconfig.yaml)
ws_config = WorkspaceConfig()
 
# Connect to Azure Sentinel with our QueryProvider and config details
qry_prov.connect(ws_config)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Test the Azure Sentinel configuration

### Viewing the Azure Sentinel workspace data schema

You can use the schema to help understand what data is available to query.<br>
The AzureSentinel QueryProvider has a "schema_tables" property that lets us get a list of tables
as well the schema (column names and data types) for each table.

<p style="border: solid; padding: 5pt"><b>Note:</b>
If you see an error here, make sure that you have run the previous cell and the authentication succeeded.
</p>

In [10]:
# Get list of tables in our Workspace with the 'schema_tables' property
print("Sample of first 10 tables in the schema")
qry_prov.schema_tables[:10]  # We are outputting only a sample of tables for brevity
                              # remove the "[:10]" to see the whole list


Sample of first 10 tables in the schema


['AACAudit',
 'AACHttpRequest',
 'AADDomainServicesAccountLogon',
 'AADDomainServicesAccountManagement',
 'AADDomainServicesDirectoryServiceAccess',
 'AADDomainServicesLogonLogoff',
 'AADDomainServicesPolicyChange',
 'AADDomainServicesPrivilegeUse',
 'AADDomainServicesSystemSecurity',
 'AADManagedIdentitySignInLogs']

## Azure Sentinel queries in MSTICPy

MSTICPy includes a number of built-in queries that you can run.<br>
You can list available queries with .list_queries() and get specific details about a query by calling it with "?" as a parameter

In [11]:
# Get a sample of available queries
print("Sample of queries")
print("=================")
print(qry_prov.list_queries()[::5])  # showing a sample - remove "[::5]" for whole list

# Get help about a query by passing "?" as a parameter
print("\nHelp for 'list_all_signins_geo' query")
print("=====================================")
qry_prov.Azure.list_all_signins_geo("?")

Sample of queries
['Azure.get_vmcomputer_for_host', 'Azure.list_azure_activity_for_account', 'AzureNetwork.az_net_analytics', 'AzureNetwork.get_heartbeat_for_ip', 'AzureSentinel.get_bookmark_by_id', 'Heartbeat.get_heartbeat_for_host', 'LinuxSyslog.all_syslog', 'LinuxSyslog.list_logon_failures', 'LinuxSyslog.sudo_activity', 'MultiDataSource.get_timeseries_decompose', 'Network.get_host_for_ip', 'Office365.list_activity_for_ip', 'SecurityAlert.list_alerts_for_ip', 'ThreatIntelligence.list_indicators_by_filepath', 'WindowsSecurity.get_parent_process', 'WindowsSecurity.list_host_events', 'WindowsSecurity.list_hosts_matching_commandline', 'WindowsSecurity.list_other_events']

Help for 'list_all_signins_geo' query
Query:  list_all_signins_geo
Data source:  AzureSentinel
Gets Signin data used by morph charts

Parameters
----------
add_query_items: str (optional)
    Additional query clauses
end: datetime (optional)
    Query end time
start: datetime (optional)
    Query start time
    (default

## Query browser

The query browser combines both of the previous functions in a scrollable<br>
and filterable list. For the selected query, it shows the required and<br>
optional parameters, together with the full text of the query.<br>

You cannot execute queries from the browser but you can copy and paste
the example shown below the help for each query.


In [12]:
qry_prov.browse_queries()

## Run some queries

### Most queries require time parameters!

Datetime strings are **painful** to type in and keep track of. 
We can use MSTICPy's `nbwidgets.QueryTime` class to define "start" and "end" for queries.

Each query provider has its own `QueryTime` instance built-in. If the query
needs "start" and "end" parameters and you do not supply them, the query
will take the time from this built-in timerange control.

In [13]:
# Open the query time control for our query provider
qry_prov.query_time

### Run a query using this time range.

Query results are returned as a [Pandas DataFrame](https://pandas.pydata.org/).
A dataframe is a tabular data structure much like a spreadsheet or
database table.

<details>
    <summary>Learn more about DataFrames...</summary>
    <p>
    <ul>
        <li>
        The <a href=https://pandas.pydata.org/ >Pandas website</a> has an extensive user guide.</li>
        <li>The <i>A Tour of Cybersec notebook features</i> has a brief introduction to common pandas operations.</li>
    </ul>
    </p>
</details>


In [14]:
# The time parameters are taken from the qry_prov time settings
# but you can override this by supplying explict "start" and "end" datetimes
signins_df = qry_prov.Azure.list_all_signins_geo()

if signins_df.empty:
    md("The query returned no rows for this time range. You might want to increase the time range")

# display first 5 rows of any results
signins_df.head() # If you have no data you will just see the column headings displayed

<IPython.core.display.Javascript object>

,TenantId,SourceSystem,TimeGenerated,ResourceId,OperationName,OperationVersion,Category,ResultType,ResultSignature,ResultDescription,DurationMs,CorrelationId,Resource,ResourceGroup,ResourceProvider,Identity,Level,Location,AlternateSignInName,AppDisplayName,AppId,AuthenticationDetails,AuthenticationMethodsUsed,AuthenticationProcessingDetails,AuthenticationRequirement,...,RiskLevelDuringSignIn,RiskState,ResourceDisplayName,ResourceIdentity,ServicePrincipalId,ServicePrincipalName,Status,TokenIssuerName,TokenIssuerType,UserAgent,UserDisplayName,UserId,UserPrincipalName,AADTenantId,UserType,FlaggedForReview,IPAddressFromResourceProvider,SignInIdentifier,SignInIdentifierType,ResourceTenantId,HomeTenantId,Type,Result,Latitude,Longitude
0,8ecf8077-cf51-4820-aadd-14040956f35d,Azure AD,2021-07-19 06:01:09.796000+00:00,/tenants/4b2462a4-bbee-495a-a0e1-f23ae524cc9c/providers/Microsoft.aadiam,Sign-in activity,1.0,SignInLogs,0,None,,0,1f7b2f6a-2a9c-4de6-80bf-098e40102b5b,Microsoft.aadiam,Microsoft.aadiam,,Bishal Das,4,IN,,Azure Portal,c44b4083-3bb0-49c1-b47d-974e53cbdf3c,"[\r\n {\r\n ""authenticationStepDateTime"": ""2021-07-19T06:01:09.7962388+00:00"",\r\n ""authe...",,"[\r\n {\r\n ""key"": ""IsCAEToken"",\r\n ""value"": ""False""\r\n }\r\n]",singleFactorAuthentication,...,none,none,Windows Azure Service Management API,797f4846-ba00-4fd7-ba43-dac1f8f63013,,,{'errorCode': 0},,AzureAD,"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.447...",Bishal Das,9600503a-5321-443d-b40e-64e8ca3d908c,bishal.das@happiestminds.com,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,Guest,None,,,,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,77428205-87ff-4048-a645-91b337240228,SigninLogs,Sucess,28.51129913330078,77.36689758300781
1,8ecf8077-cf51-4820-aadd-14040956f35d,Azure AD,2021-07-17 22:58:11.594000+00:00,/tenants/4b2462a4-bbee-495a-a0e1-f23ae524cc9c/providers/Microsoft.aadiam,Sign-in activity,1.0,SignInLogs,0,None,,0,47d913c5-8137-4945-b68b-01257a75c843,Microsoft.aadiam,Microsoft.aadiam,,On-Premises Directory Synchronization Service Account,4,US,Sync_DC01_3862ce34675f@seccxpninja.onmicrosoft.com,Microsoft Azure Active Directory Connect,cb1056e2-e479-49de-ae31-7812af012ed8,"[\r\n {\r\n ""authenticationStepDateTime"": ""2021-07-17T22:58:11.5949064+00:00"",\r\n ""authe...",,"[\r\n {\r\n ""key"": ""IsCAEToken"",\r\n ""value"": ""False""\r\n }\r\n]",singleFactorAuthentication,...,none,none,Windows Azure Active Directory,00000002-0000-0000-c000-000000000000,,,{'errorCode': 0},,AzureAD,,On-Premises Directory Synchronization Service Account,ee856d98-cecd-4dbe-8833-bdeec67847d0,sync_dc01_3862ce34675f@seccxpninja.onmicrosoft.com,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,Member,None,,Sync_DC01_3862ce34675f@seccxpninja.onmicrosoft.com,,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,SigninLogs,Sucess,38.7130012512207,-78.15899658203125
2,8ecf8077-cf51-4820-aadd-14040956f35d,Azure AD,2021-07-17 22:59:56.029000+00:00,/tenants/4b2462a4-bbee-495a-a0e1-f23ae524cc9c/providers/Microsoft.aadiam,Sign-in activity,1.0,SignInLogs,0,None,,0,f1990929-a748-460d-bd08-158853d18a51,Microsoft.aadiam,Microsoft.aadiam,,On-Premises Directory Synchronization Service Account,4,US,Sync_AADCON_a5225d32ba79@seccxpninja.onmicrosoft.com,Microsoft Azure Active Directory Connect,cb1056e2-e479-49de-ae31-7812af012ed8,"[\r\n {\r\n ""authenticationStepDateTime"": ""2021-07-17T22:59:56.029704+00:00"",\r\n ""authen...",,"[\r\n {\r\n ""key"": ""IsCAEToken"",\r\n ""value"": ""False""\r\n }\r\n]",singleFactorAuthentication,...,none,none,Windows Azure Active Directory,00000002-0000-0000-c000-000000000000,,,{'errorCode': 0},,AzureAD,,On-Premises Directory Synchronization Service Account,2235a468-ad9c-4375-8008-0a7be76994a7,sync_aadcon_a5225d32ba79@seccxpninja.onmicrosoft.com,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,Member,None,,Sync_AADCON_a5225d32ba79@seccxpninja.onmicrosoft.com,,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,SigninLogs,Su

## Customizable queries

Most built-in queries support the "add_query_items" parameter.
You can use this to append additional filters or other operations to the built-in queries.

Azure Sentinel queries use the Kusto Query Language (KQL).

<div style=border: solid; padding: 5pt"><b>Note:</b>
If this query returns too many or too few results you can change the "28"
in the query below to a smaller or larger number of days.
</div>
<br>

<details>
    <summary>Learn more about KQL query syntax...</summary>
    <p>
    <a href=https://aka.ms/kql >Kusto Query Language reference</a><br>
    </p>
</details>
<br>


In [15]:
from datetime import datetime, timedelta

qry_prov.SecurityAlert.list_alerts(
    start=datetime.utcnow() - timedelta(28),
    end=datetime.utcnow(),
    add_query_items="| summarize NumAlerts=count() by AlertName"
)

<IPython.core.display.Javascript object>

,AlertName,NumAlerts
0,Suspected brute-force attack (LDAP),1207
1,Potential Password Spray,138
2,LateralMovement_EWSCompromisedIOT-CustomGBBAnalytic,4
3,PrivilegeEscalationIOT-CustomGBBAnalytic,4
4,LateralMovement_PCNCompromisedIOT-CustomGBBAnalytic,4
...,...,...
199,Suspicious 'PowershellInjection' behavior was blocked,2
200,Anomalous account lookups,1
201,Suspected Windows Print Spooler service exploitation attempt (CVE-2021-34527 exploitation),4
202,Suspicious granting of permissions to an account,1


## Custom queries

Another way to run queries is to pass a full KQL query string to the query provider.

This will run the query against the workspace connected to above, and will return the data 
as DataFrame.


In [16]:
# Define our query
test_query = """
OfficeActivity
| where TimeGenerated > ago(1d)
| take 10
"""

# Pass that query to our QueryProvider
office_events_df = qry_prov.exec_query(test_query)
display(office_events_df.head())


<IPython.core.display.Javascript object>

,TenantId,Application,UserDomain,UserAgent,RecordType,TimeGenerated,Operation,OrganizationId,OrganizationId_,UserType,UserKey,OfficeWorkload,ResultStatus,ResultReasonType,OfficeObjectId,UserId,UserId_,ClientIP,ClientIP_,Scope,Site_,ItemType,EventSource,Source_Name,MachineDomainInfo,...,ChannelType,ChannelName,ChannelGuid,ExtraProperties,AddOnType,AddonName,TabType,Name,OldValue,NewValue,ItemName,ChatThreadId,ChatName,CommunicationType,AADGroupId,AddOnGuid,AppDistributionMode,TargetUserId,OperationScope,AzureADAppId,OperationProperties,AppId,ClientAppId,Type,_ResourceId
0,8ecf8077-cf51-4820-aadd-14040956f35d,,,,50,2021-07-22 01:03:05+00:00,MailItemsAccessed,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,Regular,100320003F8A6FC7,Exchange,Succeeded,Succeeded,,MeganB@seccxp.ninja,MeganB@seccxp.ninja,,,,,,,,,...,,,,None,,,,,,,,,,,,,,,,,"[{'Value': 'Bind', 'Name': 'MailAccessType'}, {'Value': 'False', 'Name': 'IsThrottled'}]",414a677a-e50f-46ea-b89c-aebb8a9efbe2,,OfficeActivity,
1,8ecf8077-cf51-4820-aadd-14040956f35d,,,,50,2021-07-22 01:03:05+00:00,MailItemsAccessed,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,Regular,100320003F8A6FC7,Exchange,Succeeded,Succeeded,,MeganB@seccxp.ninja,MeganB@seccxp.ninja,,,,,,,,,...,,,,None,,,,,,,,,,,,,,,,,"[{'Value': 'Bind', 'Name': 'MailAccessType'}, {'Value': 'False', 'Name': 'IsThrottled'}]",414a677a-e50f-46ea-b89c-aebb8a9efbe2,,OfficeActivity,
2,8ecf8077-cf51-4820-aadd-14040956f35d,,,,50,2021-07-22 01:03:05+00:00,MailItemsAccessed,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,Regular,100320003F8A6FC7,Exchange,Succeeded,Succeeded,,MeganB@seccxp.ninja,MeganB@seccxp.ninja,,,,,,,,,...,,,,None,,,,,,,,,,,,,,,,,"[{'Value': 'Bind', 'Name': 'MailAccessType'}, {'Value': 'False', 'Name': 'IsThrottled'}]",414a677a-e50f-46ea-b89c-aebb8a9efbe2,,OfficeActivity,
3,8ecf8077-cf51-4820-aadd-14040956f35d,,,,50,2021-07-22 01:03:05+00:00,MailItemsAccessed,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,Regular,100320003F8A6FC7,Exchange,Succeeded,Succeeded,,MeganB@seccxp.ninja,MeganB@seccxp.ninja,,,,,,,,,...,,,,None,,,,,,,,,,,,,,,,,"[{'Value': 'Bind', 'Name': 'MailAccessType'}, {'Value': 'False', 'Name': 'IsThrottled'}]",414a677a-e50f-46ea-b89c-aebb8a9efbe2,,OfficeActivity,
4,8ecf8077-cf51-4820-aadd-14040956f35d,,,,ExchangeAdmin,2021-07-22 01:06:47+00:00,Set-ConditionalAccessPolicy,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,4b2462a4-bbee-495a-a0e1-f23ae524cc9c,DcAdmin,NAMP221\Administrator (Microsoft.Office.Datacenter.Torus.PowerShellWorker),Exchange,True,True,seccxpninja.onmicrosoft.com\8837fdaa-aaeb-4432-9622-43f6f583d583,NAMP221\Administrator (Microsoft.Office.Datacenter.Torus.PowerShellWorker),NAMP221\Administrator (Microsoft.Office.Datacenter.Torus.PowerShellWorker),,,,,,,,,...,,,,None,,,,,,,,,,,,,,,,,None,,,OfficeActivity,


<details>
    <summary>Learn more about MSTICPy pre-defined queries...</summary>
    <ul>
      <li>
        <a href=https://msticpy.readthedocs.io/en/latest/data_acquisition/DataProviders.html#running-an-pre-defined-query >Running MSTICPy pre-defined queries</a>
      </li>
      <li>
        <a href=https://msticpy.readthedocs.io/en/latest/data_acquisition/DataQueries.html>MSTICPy query reference</a>
      </li>
    </ul>
</details>
<br>


---

# Testing Threat Intelligence (VirusTotal) and IP location (GeoLite2) configuration

Threat intelligence and IP location are two common enrichments that you might apply to queried data.

Let's test the VirusTotal provider with a known bad IP Address.

<details>
    <summary>Learn more...</summary>
    <p>
    </p>
    <ul>
        <li>More details are shown in the <i>A Tour of Cybersec notebook features</i> notebook</li>
        <li><a href=https://msticpy.readthedocs.io/en/latest/data_acquisition/TIProviders.html >Threat Intel Lookups in MSTICPy</a></li>
    </ul>
</details>
<br>


## Threat intel lookup


In [17]:
# Create our TI provider
ti = TILookup()

# Lookup an IP Address
ti_resp = ti.lookup_ioc("85.214.149.236", providers=["VirusTotal"])

ti_df = ti.result_to_df(ti_resp)
ti.browse_results(ti_df, severities="all")

Using Open PageRank. See https://www.domcop.com/openpagerank/what-is-openpagerank


## Geo-IP Lookup

<div style="border: solid; padding: 5pt"><b>Note:</b>
You may see the GeoLite driver downloading its database the first time you run this.
</div>
<br>
<details>
    <summary>Learn more about MSTICPy GeoIP providers...</summary>
    <p>
    <a href=https://msticpy.readthedocs.io/en/latest/data_acquisition/GeoIPLookups.html >MSTICPy GeoIP Providers</a>
    </p>
</details>
<br>


In [18]:
geo_ip = GeoLiteLookup()
raw_res, ip_entity = geo_ip.lookup_ip("85.214.149.236")
display(ip_entity[0])

Latest local Maxmind City Database present is older than 30 days. Attempting to download new database to C:\Users\Ian\.msticpy
Extraction complete. Local Maxmind city DB: C:\Users\Ian\.msticpy\GeoLite2-City.mmdb


IpAddress(Address=85.214.149.236, Location={ 'AdditionalData': {},
  'CountryCode': 'DE',
...)

---

# Conclusion

In this notebook, we've gone through the basics of installing MSTICPy and setting up configuration.
We also briefly introduced:

- QueryProviders and querying data from Azure Sentinel
- Threat Intelligence lookups using VirusTotal
- Geo-location lookups using MaxMind GeoLite2

We encourage you to run through the **A Tour of Cybersec notebook features** notebook
to get a better feel for some more of the capabilities of notebooks and MSTICPy.</br>

This notebook includes:

- more examples of queries
- visualizing your data
- brief introduction to using panda to manipulate your data.

Also try out any of the notebooks in the [Azure Sentinel Notebooks GitHub repo](https://github.com/Azure/Azure-Sentinel-Notebooks)


---

# Futher resources

 - [Jupyter Notebooks: An Introduction](https://realpython.com/jupyter-notebook-introduction/)
 - [Threat Hunting in the cloud with Azure Notebooks](https://medium.com/@maarten.goet/threat-hunting-in-the-cloud-with-azure-notebooks-supercharge-your-hunting-skills-using-jupyter-8d69218e7ca0)
 - [MSTICPy documentation](https://msticpy.readthedocs.io/)
 - [Azure Sentinel Notebooks documentation](https://docs.microsoft.com/en-us/azure/sentinel/notebooks)
 - [The Infosec Jupyterbook](https://infosecjupyterbook.com/introduction.html)
 - [Linux Host Explorer Notebook walkthrough](https://techcommunity.microsoft.com/t5/azure-sentinel/explorer-notebook-series-the-linux-host-explorer/ba-p/1138273)
 - [Why use Jupyter for Security Investigations](https://techcommunity.microsoft.com/t5/azure-sentinel/why-use-jupyter-for-security-investigations/ba-p/475729)
 - [Security Investigtions with Azure Sentinel & Notebooks](https://techcommunity.microsoft.com/t5/azure-sentinel/security-investigation-with-azure-sentinel-and-jupyter-notebooks/ba-p/432921)
 - [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/index.html)
 - [Bokeh Documentation](https://docs.bokeh.org/en/latest/)

# FAQs

---

## How can I download all Azure-Sentinel-Notebooks notebooks to my Azure ML workspace?

You can use `git` to do this from a notebook.<br>
Type the following into an empty cell and run it.

```python
!git clone https://github.com/Azure/Azure-Sentinel-Notebooks.git azure-sentinel-nb
```

This will create a copy of the GitHub repo contents in the "azure-sentinel-nb" folder
of your user folder.

Copy the notebooks that you want to use from this folder to your working directory. This will allow you to update the notebooks easily in the future.

The command to update the notebooks from GitHub is

```python
!cd azure-sentinel-nb && git pull
```


---

## Can I install MSTICPy by default on a new AML compute

We have created an AML compute install script and put it in the HowTo folder
on the Azure-Sentinel-Notebooks repo.
The script is here:
[aml-compute-setup.sh](https://github.com/Azure/Azure-Sentinel-Notebooks/master/HowTos/aml-compute-setup.sh)

1. Download the script to your local computer.
2. Add a new Compute instance

<img src="https://github.com/Azure/Azure-Sentinel-Notebooks/raw/master/images/AML_compute_create.png" height="300px">

3. In the **Advanced** section enable **Provision with setup script**
   and browse to the location of the downloaded script.

<img src="https://github.com/Azure/Azure-Sentinel-Notebooks/raw/master/images/AML_compute_script.png" height="300px">

4. Click the create button to create the instance

You can view a log of the custom installation in the `/tmp/aml-setup-msticpy.log` log.



---

## I see error "Runtime dependency of PyGObject is missing" when I load a query provider

```python
qry_prov = QueryProvider("AzureSentinel")
```

Warning details

```text
Runtime dependency of PyGObject is missing.
Depends on your Linux distro, you could install it system-wide by something like:
    sudo apt install python3-gi python3-gi-cairo gir1.2-secret-1
If necessary, please refer to PyGObject's doc:
https://pygobject.readthedocs.io/en/latest/getting_started.html
Traceback (most recent call last):
  File "/anaconda/envs/azureml_py36/lib/python3.6/site-packages/msal_extensions/libsecret.py", line 21, in <module>
    import gi  # https://github.com/AzureAD/microsoft-authentication-extensions-for-python/wiki/Encryption-on-Linux
ModuleNotFoundError: No module named 'gi'
```

This is due to a missing Python dependency `pygobject`.
If you created your compute instance using the script mentioned in the previous
question, this automatically installs pygobject in all notebook/Anaconda environments
on the compute.

You can also fix this by running the following from a notebook:

```python
!conda install --yes -c conda-forge pygobject
```


---

## MSTICPy and other packages do not install properly when switching between the Python 3.6 or 3.8 Kernels

We've seen this happen intermittently. Using `!pip install pkg` will install correctly in one<br>
environment but then does not install in the other. The package cannot be imported<br>
and used in the second environment.

Avoid using `!pip install...` to install packages in AML notebooks.
Instead, use one of the following options:

1. Use the %pip line magic within a notebook.

```Python
    %pip install --upgrade msticpy
```

2. Install from a terminal.<br>
   Open a terminal in Azure ML notebooks and run the following commands:

``` bash
    conda activate azureml_py38
    pip install --upgrade msticpy
```

Close the terminal and restart the kernel.

---

## My user account/credentials do not get cached between notebook runs

You can force caching of credentials for a session by authenticating using Azure CLI.

In an empty cell type and execute the following:

```python
!az login
```

You should see the following prompt (in barely readable text if you are using the default light theme):

```
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code AMEE5STGD to authenticate.
```

Select and copy the 9 character token and click on the devicelogin URL.
Paste the token into the dialog at that page and proceed with the login as prompted.

When login has successfully completed you should see the following output:

```
Subscription 74bd0f79-3e18-4be4-90c5-0c46007ab514 'Sample subscription' can be accessed from tenants 2a0b016c-36fa-4569-974e-37b05956597c(default) and 45775191-e432-4364-a1cd-b97da6816cb4. To select a specific tenant when accessing this subscription, use 'az login --tenant TENANT_ID'.
The following tenants don't contain accessible subscriptions. Use 'az login --allow-no-subscriptions' to have tenant level access.
e8f78f0f-3f2b-4342-a36c-7e91db929057 'foo'
0aa0847b-6374-4acc-a76c-a09e92a84c56 'bar'
[
  {
    "cloudName": "AzureApp",
    "homeTenantId": "2a0b016c-36fa-4569-974e-37b05956597c",
    "id": "53b762ad-56f2-4c8f-b7e9-5eab95793ba5",
    "isDefault": true,
    "managedByTenants": [
    ....
```
